# CVAI Exercise Digital Photography 2

In this exercise we will continue to carry out some of the image manipulation that are often done as part of the processing in the camera.


In [ ]:
import skimage
import numpy as np

# for displaying images in jupyter
import matplotlib as mpl
from matplotlib import pyplot as plt

# this makes the images a bit larger
mpl.rcParams['figure.dpi']= 200
plt.rcParams['figure.figsize'] = [5,4]

# plots directly in the notebook
%matplotlib inline 

# if you have a high-dpi monitor
%config InlineBackend.figure_format = 'retina' 

path = '/exchange/cvai/images/'

## Exercise 1: White balance

We have looked at white balance in the lecture. The problem that images taken under different lighting conditions do not look the same on a photograph. When we see the scene with our own eyes, the brain will adjust so that we will still recognise the colors. However, when looking at an image that will not be done.

We will load different example images taken from the wikipedia page about color balance: https://en.wikipedia.org/wiki/Color_balance. The images are by Alex1ruff - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=48366925


In [ ]:
# images can be read using imread, this will result in a 3 channel uint8 image
image_neutral = skimage.io.imread(path + '1080px-Wb_girl_neutral.jpg')
image_tungsten = skimage.io.imread(path + '1080px-Wb_girl_tungsten.jpg')
image_cloudy = skimage.io.imread(path + '1080px-Wb_girl_cloudy.jpg')

plt.rcParams['figure.figsize'] = [15,15]
plt.subplot(3,1,1)
plt.imshow(image_neutral)
plt.subplot(3,1,2)
plt.imshow(image_tungsten)
plt.subplot(3,1,3)
plt.imshow(image_cloudy)

We will try do adapt the blueish tungsten image first. As the image contains a color checker, we can use that to find the ground truth. The color checker is the following one from X-Rite:
https://www.xrite.com/categories/calibration-profiling/colorchecker-classic

According to that website, the grey values in the lower 6 squares are approximately gray values with the values 241, 202, 163, 121, 82, 48. I.e. the brightest square has the color (241, 241, 241).

We would like to extract and average a 20x20 pixel area from each of the 6 squares. You might want to open the image in an image processing program to better find the coordinates and display the found squares to verify. Remember that numpy uses the first coordinate along the y axis.

In [ ]:
white_patch = image_tungsten[0:20, 0:20,:]
grey_1_patch = image_tungsten[0:20, 0:20,:]
grey_2_patch = image_tungsten[0:20, 0:20,:]
grey_3_patch = image_tungsten[0:20, 0:20,:]
grey_4_patch = image_tungsten[0:20, 0:20,:]
black_patch = image_tungsten[0:20, 0:20,:]

# add the correct coordinates to extract the patches

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
plt.subplot(3,2,1)
plt.imshow(white_patch)
plt.subplot(3,2,2)
plt.imshow(grey_1_patch)
plt.subplot(3,2,3)
plt.imshow(grey_2_patch)
plt.subplot(3,2,4)
plt.imshow(grey_3_patch)
plt.subplot(3,2,5)
plt.imshow(grey_4_patch)
plt.subplot(3,2,6)
plt.imshow(black_patch)

As we can see, the patches are quite blueish. Now we should calculate the average color of the patches so that we can compare them to the real values. You might have to check numpy how to average along more than one axis.

In [ ]:
# calculate the average values of the 6 patches

# initialize variables 
white_tungsten = [0, 0, 0]
grey_1_tungsten = [0, 0, 0]
grey_2_tungsten = [0, 0, 0]
grey_3_tungsten = [0, 0, 0]
grey_4_tungsten = [0, 0, 0]
black_tungsten = [0, 0, 0]

# calculate the colors correctly

# YOUR CODE HERE
raise NotImplementedError()

print(white_tungsten, grey_1_tungsten, grey_2_tungsten, grey_3_tungsten, grey_4_tungsten, black_tungsten)

In [ ]:
np.testing.assert_allclose(white_tungsten,[223, 237, 252], atol=3.0)
np.testing.assert_allclose(grey_1_tungsten,[192, 215, 246], atol=3.0)
np.testing.assert_allclose(grey_2_tungsten,[150, 181, 227], atol=3.0)
np.testing.assert_allclose(grey_3_tungsten,[84, 127, 178], atol=3.0)
np.testing.assert_allclose(grey_4_tungsten,[41, 77, 119], atol=3.0)
np.testing.assert_allclose(black_tungsten,[15, 33, 57], atol=3.0)

### Color Correction

We now want to implement the color correction. One simple solution is to scale the color channels linearly in RGB so that the correct color is achieved for a given reference color.

Write a function that takes an image, a reference color, the same color as it occurs in the image (i.e. what we have measured) and calculates the color corrected image.


In [ ]:
def color_correction(image: np.ndarray, ref_color, ref_color_measured) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError()
            

In [ ]:
# call the correction with one of the gray values (you can also try some of the other color, grey seems to work better
# than white as it will give a stronger correction )
image_corrected = color_correction(image_tungsten, [163,163,163],grey_2_tungsten)
assert image_corrected is not None

In [ ]:
plt.rcParams['figure.figsize'] = [14,10]
plt.subplot(1,2,1)
plt.imshow(image_corrected)
plt.subplot(1,2,2)
plt.imshow(image_tungsten)

The image does not yet look completely like the neutral image, but much of the blueish color has gone the the image looks much more realistic.

You can try to correct the other (cloudy) image the same way.

## Exercise 2: HDR Imaging

In this exercise we will look at creating an HDR image out of images taken with different exposure settings. We will first use the image of the St. Louis arc which are available at Wikimadia Commons under the following license:

Kevin McCoy, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons

The images have been downsampled from the original size for faster processing. Lets first load and display the 4 images.

In [ ]:
image_0 = skimage.io.imread(path + '960px-StLouisArchMultExpEV-4.72.JPG')
image_1 = skimage.io.imread(path + '960px-StLouisArchMultExpEV-1.82.JPG')
image_2 = skimage.io.imread(path + '960px-StLouisArchMultExpEV+1.18.JPG')
image_3 = skimage.io.imread(path + '960px-StLouisArchMultExpEV+4.09.JPG')

In [ ]:
plt.rcParams['figure.figsize'] = [15,15]
plt.subplot(2,2,1)
plt.imshow(image_0)
plt.subplot(2,2,2)
plt.imshow(image_1)
plt.subplot(2,2,3)
plt.imshow(image_2)
plt.subplot(2,2,4)
plt.imshow(image_3)

Write a function that combines the images to produce one with larger dynamic range. In the resulting image, you will want to see some details from the dark areas without having the bright areas overexposed.

As outlined in the course, you might only want to use the pixels in a specific location from some images, but not from others. Try to use only linear combination for the first approach, even as the images are jpeg.

Is there much difference if you mask out pixels from some of the images or not?


In [ ]:
# save your resulting image in this variable
linear_hdr = None
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert linear_hdr is not None
plt.imshow(linear_hdr)

As you can see, there is one problem with the images, they do not seem to be fully aligned. We will look how to correct this problem in the next lecture.

This will be all for the exercise.

Please feel free to experiment a bit more with the images.